# BIDDING STATERGIES

## Approach 1

In this approach, we just bid the predicted demand after subtracting the predicted solar output at the predicted market price.

In [1]:
import numpy as np
import pandas as pd
from scorer import bill_maker

In [2]:
demand=pd.read_csv("Prediction/demand.csv", header=None)
market_price=pd.read_csv("Prediction/market_price.csv", header=None)
solar_output=pd.read_csv("Prediction/solar_output.csv", header=None)

In [3]:
bid_quantity_pred=pd.DataFrame(demand-solar_output)

In [4]:
bid_price_pred=pd.DataFrame(market_price)

### Comparing prediction with Oracle

In [5]:
demand_oracle=pd.read_csv("Processed Data/Demand_Val_pred.csv", header=None)
market_price_oracle=pd.read_csv("Processed Data/Price_Val_pred.csv", header=None)
solar_output_oracle=pd.read_csv("Processed Data/Solar_Val_pred.csv", header=None)

In [6]:
bid_quantity_oracle=pd.DataFrame(demand_oracle - solar_output_oracle)

In [7]:
bid_price_oracle=pd.DataFrame(market_price_oracle)

In [8]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

In [9]:
daily_bill_oracle,total_bill_oracle=bill_maker(bid_price_oracle,bid_quantity_oracle)

In [10]:
total_bill_pred

383320.09645041835

In [11]:
total_bill_oracle

418580.69730000006

### Saving prediction to output

In [12]:
bid_quantity_pred.to_csv("Output/BidQuantity.csv", index=False, header=False)
bid_price_pred.to_csv("Output/BidPrice.csv", index=False, header=False)

## Approach 2

In this approach, we try to find the optimal margin that should be added to market price so that we avoid loosing the bid. We try to find this margin by considering the full training data.

In [13]:
demand_actual_train=pd.read_csv("Processed Data/Demand_Train.csv", header=None)
market_price_train=pd.read_csv("Processed Data/Price_Train.csv", header=None)
market_price_train_oracle=pd.read_csv("Processed Data/Price_Train_pred.csv", header=None)
solar_output_train=pd.read_csv("Processed Data/Solar_Train.csv", header=None)

In [14]:
pd.DataFrame(market_price_train.describe()[1:2].mean()).describe()

,0
count,24.000000
mean,2.904113
std,0.950572
min,1.577847
25%,2.067374
50%,3.015271
75%,3.423165
max,4.616106


In [15]:
pd.DataFrame(market_price.describe()[1:2].mean()).describe()

,0
count,24.000000
mean,2.989362
std,1.005487
min,1.603741
25%,2.102923
50%,3.115149
75%,3.523238
max,4.831434


In [16]:
pd.DataFrame(market_price.describe()[1:2].mean()).describe().loc[["mean"]]-pd.DataFrame(market_price_train.describe()[1:2].mean()).describe().loc[["mean"]]

,0
mean,0.085249


From the above difference of mean, we can see that mean of predicted market price is greater than that of the training data, Hence we subtract this mean from the predicted market price.

In [17]:
bid_price_pred=pd.DataFrame(market_price - 0.085249)

In [18]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [19]:
total_bill_pred

473192.10357479547

In [20]:
total_bill_oracle

418580.69730000006

We can also find hour wise mean for the training data and correspondingly add/subtract from our predicted market price.

In [21]:
hourly_mean=(market_price.describe().loc[["mean"]] - market_price_train.describe().loc[["mean"]]).mean()

In [22]:
bid_price_pred=pd.DataFrame(market_price - hourly_mean)

In [23]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [24]:
total_bill_pred

454251.62216230325

In [25]:
total_bill_oracle

418580.69730000006

We can also subtract the 25th percentile of the difference of price predicted by oracle and actual price when the predicted price by Oracle is less than Actual price (the case when we loose the bid) to ensure that 75% the we win it.

In [26]:
margin_25=(market_price_train_oracle - market_price_train)[(market_price_train_oracle - market_price_train)<0].describe().loc[['25%']].mean()

In [27]:
bid_price_pred=pd.DataFrame(market_price - margin_25)

In [28]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [29]:
total_bill_pred

309519.2878948272

In [30]:
total_bill_oracle

418580.69730000006

### Saving prediction to output

In [31]:
bid_quantity_pred.to_csv("Output/BidQuantity.csv", index=False, header=False)
bid_price_pred.to_csv("Output/BidPrice.csv", index=False, header=False)

## Approach 3

In this approach, we find the appropriate margin by considering the specific 50 days from the previous 2 years.

In [32]:
market_price_train[-365:-365 + 50].values

array([[ 1.55,  1.35,  1.35, ...,  2.4 ,  2.22,  1.35],
       [ 1.55,  1.35,  1.27, ...,  2.29,  1.89,  1.01],
       [ 1.95,  1.98,  1.93, ...,  2.9 ,  2.31,  1.28],
       ..., 
       [ 1.3 ,  1.3 ,  1.3 , ...,  2.91,  2.45,  1.42],
       [ 1.3 ,  1.3 ,  1.3 , ...,  2.63,  2.36,  1.4 ],
       [ 1.35,  1.19,  1.19, ...,  3.66,  3.13,  1.47]])

In [33]:
market_price_train[-365*2:-365*2 + 50].values

array([[ 1.97,  1.58,  1.36, ...,  2.19,  1.95,  1.81],
       [ 1.62,  1.35,  1.38, ...,  2.4 ,  2.3 ,  1.38],
       [ 1.57,  1.4 ,  1.3 , ...,  2.29,  1.93,  1.01],
       ..., 
       [ 1.76,  1.65,  1.21, ...,  2.57,  2.08,  1.03],
       [ 1.31,  1.34,  1.32, ...,  2.92,  2.56,  1.5 ],
       [ 1.32,  1.3 ,  1.34, ...,  2.69,  2.45,  1.42]])

We find the average market price by the considering the 50 day period from the two previous years.

In [34]:
market_price_avg = (market_price_train[-365:-365 + 50].values + market_price_train[-365*2:-365*2 + 50].values)/2

In [35]:
market_price_avg=pd.DataFrame(market_price_avg)

In [36]:
market_price_avg.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1.760,1.465,1.355,1.365,1.375,2.055,3.795,4.625,4.645,4.795,...,2.295,2.570,2.83,3.030,3.235,3.005,3.015,2.295,2.085,1.580
1,1.585,1.350,1.325,1.175,1.160,2.205,3.665,4.725,4.980,5.765,...,3.290,2.500,2.74,3.000,3.360,2.935,2.975,2.345,2.095,1.195
2,1.760,1.690,1.615,1.425,1.740,2.965,4.715,5.640,5.840,6.650,...,3.740,2.995,3.02,3.340,3.590,2.815,2.945,2.595,2.120,1.145
3,2.150,2.145,2.125,2.105,2.440,2.395,4.645,5.725,6.095,5.855,...,3.550,3.675,3.48,3.565,3.685,3.135,3.150,2.945,2.245,1.425
4,2.295,2.325,2.330,2.290,2.295,2.375,4.575,5.275,5.725,5.770,...,3.720,3.895,3.72,3.370,3.690,3.320,3.225,2.665,2.340,1.595


We model margin through subtracting the 25 percentile value of the data when predicted market price is less than average market price. 

In [37]:
margin_25=(market_price - market_price_avg)[(market_price - market_price_avg)<0].describe().loc[['25%']].mean()

In [38]:
bid_price_pred=pd.DataFrame(market_price - margin_25)

In [39]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [40]:
total_bill_pred

320438.20609907928

In [41]:
total_bill_oracle

418580.69730000006

We can also add the absolute margin to the predicted market price.

In [42]:
abs_margin=(market_price - market_price_avg).abs().describe().loc[["mean"]].mean()

In [43]:
bid_price_pred=pd.DataFrame(market_price + abs_margin)

In [44]:
daily_bill_pred,total_bill_pred=bill_maker(bid_price_pred,bid_quantity_pred)

### Comparing prediction with Oracle

In [45]:
total_bill_pred

312821.5886400243

In [46]:
total_bill_oracle

418580.69730000006

### Saving prediction to output

In [47]:
bid_quantity_pred.to_csv("Output/BidQuantity.csv", index=False, header=False)
bid_price_pred.to_csv("Output/BidPrice.csv", index=False, header=False)